In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Fake News Detection - Feature Analysis\n",
    "\n",
    "This notebook analyzes the features extracted from text data for fake news detection. We'll explore:\n",
    "1. Feature extraction and visualization\n",
    "2. Feature importance analysis\n",
    "3. Model performance comparison\n",
    "4. Correlation analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import sys\n",
    "import os\n",
    "sys.path.append(os.path.abspath('../src'))\n",
    "\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "from sklearn.feature_extraction.text import TfidfVectorizer\n",
    "from sklearn.ensemble import RandomForestClassifier\n",
    "from sklearn.decomposition import PCA\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "\n",
    "from data_processor import FakeNewsDataset\n",
    "from features import FeatureExtractor\n",
    "from models import EnhancedClassifier"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load and Prepare Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "dataset = FakeNewsDataset('../data')\n",
    "texts = dataset.texts\n",
    "labels = dataset.labels\n",
    "\n",
    "print(f\"Total samples: {len(texts)}\")\n",
    "print(f\"Fake news samples: {sum(labels)}\")\n",
    "print(f\"Real news samples: {len(labels) - sum(labels)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Extract Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "feature_extractor = FeatureExtractor()\n",
    "features = feature_extractor.extract_features(texts)\n",
    "\n",
    "print(\"Extracted features:\")\n",
    "for feature_name, feature_values in features.items():\n",
    "    print(f\"- {feature_name}: {len(feature_values)} samples\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Feature Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def plot_feature_distribution(feature_name, values, labels):\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    sns.histplot(data=pd.DataFrame({\n",
    "        feature_name: values,\n",
    "        'label': ['Fake' if l == 1 else 'Real' for l in labels]\n",
    "    }), x=feature_name, hue='label', kde=True)\n",
    "    plt.title(f'Distribution of {feature_name}')\n",
    "    plt.show()\n",
    "\n",
    "for feature_name, feature_values in features.items():\n",
    "    plot_feature_distribution(feature_name, feature_values, labels)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Feature Importance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "X = np.column_stack(list(features.values()))\n",
    "y = np.array(labels)\n",
    "\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n",
    "\n",
    "rf = RandomForestClassifier(n_estimators=100, random_state=42)\n",
    "rf.fit(X_train, y_train)\n",
    "\n",
    "feature_importance = pd.DataFrame({\n",
    "    'feature': list(features.keys()),\n",
    "    'importance': rf.feature_importances_\n",
    "}).sort_values('importance', ascending=False)\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.barplot(data=feature_importance, x='importance', y='feature')\n",
    "plt.title('Feature Importance')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Correlation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "feature_df = pd.DataFrame(features)\n",
    "correlation_matrix = feature_df.corr()\n",
    "\n",
    "plt.figure(figsize=(12, 10))\n",
    "sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)\n",
    "plt.title('Feature Correlation Matrix')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Dimensionality Reduction and Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "scaler = StandardScaler()\n",
    "X_scaled = scaler.fit_transform(X)\n",
    "\n",
    "pca = PCA(n_components=2)\n",
    "X_pca = pca.fit_transform(X_scaled)\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.scatterplot(\n",
    "    x=X_pca[:, 0],\n",
    "    y=X_pca[:, 1],\n",
    "    hue=['Fake' if l == 1 else 'Real' for l in labels],\n",
    "    alpha=0.6\n",
    ")\n",
    "plt.title('PCA of Features')\n",
    "plt.xlabel('First Principal Component')\n",
    "plt.ylabel('Second Principal Component')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Model Performance Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def evaluate_model(model, X_train, X_test, y_train, y_test):\n",
    "    model.fit(X_train, y_train)\n",
    "    y_pred = model.predict(X_test)\n",
    "    \n",
    "    print(classification_report(y_test, y_pred))\n",
    "    \n",
    "    plt.figure(figsize=(8, 6))\n",
    "    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')\n",
    "    plt.title('Confusion Matrix')\n",
    "    plt.show()\n",
    "\n",
    "print(\"Random Forest Classifier:\")\n",
    "evaluate_model(rf, X_train, X_test, y_train, y_test)\n",
    "\n",
    "print(\"\\nEnhanced Classifier:\")\n",
    "enhanced_model = EnhancedClassifier()\n",
    "evaluate_model(enhanced_model, X_train, X_test, y_train, y_test)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}